In [4]:
import os
import numpy as np
import pandas as pd
import cv2
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [5]:
import tensorflow as tf
import pynvml
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Check if TensorFlow is using the GPU
if tf.test.is_gpu_available():
    print("TensorFlow is using the GPU")
    
    # Initialize the pynvml library
    pynvml.nvmlInit()
    
    # Get the number of GPU devices
    num_gpus = pynvml.nvmlDeviceGetCount()
    
    # Iterate over GPU devices
    for i in range(num_gpus):
        # Get the device identifier
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        # Get the full GPU name
        gpu_name = pynvml.nvmlDeviceGetName(handle)
        print("GPU Name:", gpu_name)
        
    # Shutdown the pynvml library
    pynvml.nvmlShutdown()
else:
    print("TensorFlow is not using the GPU")

TensorFlow is using the GPU
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [6]:
video_path = 'GAZE_dataset/video'
original_annotation_path = 'GAZE_dataset/annotation_cleaned/'
my_annotation_path = 'GAZE_dataset/annotations.csv'
patches_path = 'GAZE_dataset/preprocess/video_data/'
pca_dir = "GAZE_dataset/preprocess/fitted_PCA"

In [7]:
def load_pickle(path):
    with open(path, 'rb') as f:
        pickled = pickle.load(f)
    return pickled

def save_pickle(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

## Face annotations

loop each author annotations to creat my annotations

In [ ]:
bounding_box_data = []

for i in range(1, 302):
    each_annotation = f'NewAnt_{i}.txt'
    annotation_path = os.path.join(original_annotation_path, each_annotation)
    video_name = f'{i}.mp4'
    try:
        with open(annotation_path, 'r') as f:
            file_lines = f.readlines()
    except FileNotFoundError:
        print(f'File {annotation_path} not found, the annotation is lost')
        continue
    for line in file_lines:
        if line.strip():
            data = line.strip().split(' ')
            label = data[9]
            if len(data) == 10:
                gaze_direction = None
                event_attribute = None
                atomic_attribute = None
            elif len(data) == 13:
                gaze_direction = data[-1]
                if gaze_direction == 'P1': gaze_direction = f'{label}, Person1'
                elif gaze_direction == 'P2': gaze_direction = f'{label}, Person2'
                elif gaze_direction == 'P3': gaze_direction = f'{label}, Person3'
                elif gaze_direction == 'P4': gaze_direction = f'{label}, Person4'
                elif gaze_direction == 'O1': gaze_direction = f'{label}, Object1'
                elif gaze_direction == 'O2': gaze_direction = f'{label}, Object2'
                elif gaze_direction == 'O3': gaze_direction = f'{label}, Object3'
                elif gaze_direction == 'O4': gaze_direction = f'{label}, Object4'
                elif gaze_direction == 'O5': gaze_direction = f'{label}, Object5'
                elif gaze_direction == 'NA': gaze_direction = None
                event_attribute = data[10]
                atomic_attribute = data[11]
                
            frame_data = {'video_name': video_name,
                          'frame': int(data[5])+1,
                          'label_name': data[9],
                          'left': int(data[1]),
                          'top' : int(data[2]),
                          'right': int(data[3]),
                          'bottom': int(data[4]),
                          'gaze_direction': gaze_direction,
                          'event_attribute': event_attribute,
                          'atomic_attribute': atomic_attribute,
                          }
            bounding_box_data.append(frame_data)

In [ ]:
annotations = pd.DataFrame(bounding_box_data)
annotations

In [ ]:
annotations.to_csv(my_annotation_path, index=False)

## CROP OUT IMAGES PATCHES FROM VIDEOS


In [ ]:
annotations = pd.read_csv(my_annotation_path)
annotations

In [ ]:
# Iterate over each unique video in the DataFrame
for video_name in tqdm(annotations['video_name'].unique()):
    # print("Processing video:", video_name)
    # Fetch all frames annotations in this video
    save_path = os.path.join(patches_path, video_name)
    os.makedirs(save_path, exist_ok=True)
    video = cv2.VideoCapture(os.path.join(video_path, video_name))
    video_annotations = annotations[annotations['video_name'] == video_name]
    patches = []
    annotations_dict = {'labels': [], 'frame_numbers': [], 
                        'left': [], 'right': [], 'top': [], 'bottom': [],
                        'gaze_direction': [], 'event_attribute': [], 'atomic_attribute': []}
    
    for current_frame in range(1, video_annotations['frame'].iloc[-1]+1):  # in the range of number of frames
        successful_read, frame = video.read()
        if successful_read:
            # Filter annotations for the current frame
            frame_annotations = video_annotations[video_annotations['frame'] == current_frame]
            if not frame_annotations.empty:
                for _, entity in frame_annotations.iterrows():
                    patches.append(frame[int(entity['top']):int(entity['bottom']),int(entity['left']):int(entity['right'])])
                    annotations_dict['labels'].append(entity['label_name'])
                    annotations_dict['frame_numbers'].append(current_frame)
                    annotations_dict['left'].append(int(entity['left']))
                    annotations_dict['right'].append(int(entity['right']))
                    annotations_dict['top'].append(int(entity['top']))
                    annotations_dict['bottom'].append(int(entity['bottom']))
                    annotations_dict['gaze_direction'].append(entity['gaze_direction'])
                    annotations_dict['event_attribute'].append(entity['event_attribute'])
                    annotations_dict['atomic_attribute'].append(entity['atomic_attribute'])
                    
        else:
            raise ValueError(f"Unsuccessful read frame {current_frame} of {video_name}")
    save_pickle(patches, os.path.join(save_path, 'patches'))
    annotation_df = pd.DataFrame(annotations_dict)
    save_pickle(annotation_df, os.path.join(save_path, 'annotations'))


In [ ]:
 def visualize_patches(num_patches_to_display=10):
    # Get a list of all subdirectories in the patches_output_path directory
    videos = [d for d in os.listdir(patches_path)]
    import random
    # Randomly select one of the subdirectories
    selected_subdir = random.choice(videos)
    selected_path = os.path.join(patches_path, selected_subdir)
    
    patches = load_pickle(os.path.join(selected_path, 'patches'))
    annot = load_pickle(os.path.join(selected_path, 'annotations'))
    
    # Display each patch with its corresponding labels
    print(len(patches))
    for i, patch in enumerate(patches):
        print(annot.loc[i, 'frame_numbers'])
        print(annot.loc[i, 'labels'])
        print(annot.loc[i, 'gaze_direction'])
        plt.imshow(cv2.cvtColor(patch, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        if i+2> num_patches_to_display:
            break


In [ ]:
visualize_patches(5)

## VGG FEATURES

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.models import Model

In [ ]:
def reshape_patches(x):
    temp = np.expand_dims(x, axis=0)
    temp2 = preprocess_input(smart_resize(temp, (224,224)))
    return temp2[0]

In [ ]:
base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [ ]:
restart = input('Do you want to reparse the input videos? (y/n)')

In [ ]:
batch_size = 32  # Adjust this batch size based on your memory capacity

for video in tqdm(os.listdir(patches_path)):
    video_dir = os.path.join(patches_path, video)
    patch_dir = os.path.join(video_dir, "patches")
    out_dir = os.path.join(video_dir, "VGG19_patches")
    if not os.path.exists(out_dir) or restart == 'y':
        patches = load_pickle(patch_dir)
        y_total = []
        
        for i in range(0, len(patches), batch_size):
            batch_patches = patches[i:i + batch_size]
            x_batch = [reshape_patches(patch) for patch in batch_patches]
            x_batch = np.array(x_batch)
            y_batch = model.predict(x_batch, verbose=0)
            y_total.extend(y_batch)
        
        y_total = np.array(y_total)
        save_pickle(y_total, out_dir)
        print(f"VGG19 patches saved to {video_dir}")


## Fit PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
def extract_all_vggfeatures():
    all_features = []
    for video in tqdm(os.listdir(patches_path)):
        patch_dir = os.path.join(patches_path, video, "VGG19_patches")
        all_features.extend(load_pickle(patch_dir))

    all_features = np.array(all_features)
    print(all_features.shape)
    return all_features


def fit_pca(all_features):
    pca = PCA(n_components=20)
    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(all_features)
    pca.fit(all_features_scaled)
    return pca, scaler

In [ ]:
pca, scaler = fit_pca(extract_all_vggfeatures())

In [ ]:
save_pickle(pca, os.path.join(pca_dir, "pca"))
save_pickle(scaler, os.path.join(pca_dir, "scaler"))

## PCA on VGG features

In [ ]:
pca = load_pickle(os.path.join(pca_dir, "pca"))
scaler = load_pickle(os.path.join(pca_dir, 'scaler'))

In [ ]:
for video in tqdm(os.listdir(patches_path)):
    video_dir = os.path.join(patches_path, video)
    patch_dir = os.path.join(video_dir, "VGG19_patches")
    vgg_features = load_pickle(patch_dir)
    scaled_features = scaler.transform(vgg_features)
    pca_features = pca.transform(scaled_features)
    save_pickle(pca_features, os.path.join(video_dir, "pca_features"))   

## Node Features


In [ ]:
for video in tqdm(os.listdir(patches_path)):
    video_dir = os.path.join(patches_path, video)
    pca_features = load_pickle(os.path.join(video_dir, "pca_features"))
    video_annot = load_pickle(os.path.join(video_dir, "annotations"))
    new_features = []
    for i, patch_feature in enumerate(pca_features):
        new_feature = np.append(patch_feature, [video_annot['top'][i], video_annot['bottom'][i], video_annot['left'][i], video_annot['right'][i]])
        new_feature = np.append(new_feature, [0] if 'Person' in video_annot['labels'][i] else [1])
        assert len(new_feature) == 25
        new_features.append(new_feature)
    video_annot['features'] = new_features
    save_pickle(video_annot, os.path.join(video_dir, "annotations"))

In [ ]:
video_annot

## Split into Sequences

In [ ]:
def split_sequences(dir, df):
    splits = []
    labels = []
    start_index = 0
    
    # Get the unique frame numbers
    unique_frames = df['frame_numbers'].unique()
    
    # Create a pair for the first frame, handling NaNs
    frame_labels = df[df['frame_numbers'] == unique_frames[0]]['labels'].tolist()
    frame_events = df[df['frame_numbers'] == unique_frames[0]]['event_attribute'].tolist()
    frame_events = [None if isinstance(x, float) else x for x in frame_events]
    previous_pairs = dict(zip(frame_labels, frame_events))

    sequences = []
    one_sequence = []
    frame_features = df[df['frame_numbers'] == unique_frames[0]]['features'].tolist()
    combined_tuple = list(zip(frame_labels, frame_events))
    frame_matrix = np.empty((2, len(frame_labels)), dtype=object)
    for j in range(len(frame_labels)):
        frame_matrix[0, j] = frame_features[j]
        frame_matrix[1, j] = combined_tuple[j]
    one_sequence.append(frame_matrix)
    
    for i in range(1, len(unique_frames)):
        frame = unique_frames[i]
        # Create a pair for the current frame, handling NaNs        
        frame_labels = df[df['frame_numbers'] == frame]['labels'].tolist()
        frame_events = df[df['frame_numbers'] == frame]['event_attribute'].tolist()
        frame_events = [None if isinstance(x, float) else x for x in frame_events]
        current_pairs = dict(zip(frame_labels, frame_events))
        
        frame_features = df[df['frame_numbers'] == frame]['features'].tolist()
        combined_tuple = list(zip(frame_labels, frame_events))
        frame_matrix = np.empty((2, len(frame_labels)), dtype=object)
        for j in range(len(frame_labels)):
            frame_matrix[0, j] = frame_features[j]
            frame_matrix[1, j] = combined_tuple[j]
        

        # Check if there's any difference in pairs
        if current_pairs != previous_pairs:
            labels.append(previous_pairs)
            sequences.append(np.array(one_sequence, dtype=object))
            one_sequence = [frame_matrix]
            # Update the previous pairs
            previous_pairs = current_pairs
            splits.append(df[df['frame_numbers'].isin(unique_frames[start_index:i])])
            start_index = i
        else:
            one_sequence.append(frame_matrix)

    # Append the last segment
    sequences.append(np.array(one_sequence, dtype=object))
    labels.append(previous_pairs)
    splits.append(df[df['frame_numbers'].isin(unique_frames[start_index:])])
    # Save all the sequences
    for i, split in enumerate(splits):
        save_pickle(split, os.path.join(dir, f"sequence_{i}"))
    save_pickle(sequences, os.path.join(dir,'stupid_sequences'))
    save_pickle(labels, os.path.join(dir, f"labels"))

In [ ]:
for video in tqdm(os.listdir(patches_path)):
    video_dir = os.path.join(patches_path, video)
    video_annot = load_pickle(os.path.join(video_dir, "annotations"))
    split_sequences(video_dir, video_annot)

## Comparing preprocessed results

In [ ]:
test = load_pickle('GAZE_dataset/preprocess/video_data/1.mp4/sequences')
test_label = load_pickle('GAZE_dataset/preprocess/video_data/1.mp4/labels')

In [ ]:
test

281， 222， 329， 221
rig, left, bot, top
top, bot, lef, rig

In [ ]:
test_label

In [ ]:
manasi_test = load_pickle('GAZE_dataset/Manasi_preprocessed_pickles/processed_Mar25_0')

In [ ]:
# video names
manasi_test.keys()

In [ ]:
# each video dict
manasi_test['1'].keys()

In [ ]:
manasi_test['1']['labels']

In [ ]:
# len is the sequence number, this video only has 1 sequence
type(manasi_test['1']['graph_dicts']), len(manasi_test['1']['graph_dicts'])

In [ ]:
# here is the number of frames in this sequence
type(manasi_test['1']['graph_dicts'][0]), len(manasi_test['1']['graph_dicts'][0])

In [ ]:
type(manasi_test['1']['graph_dicts'][0][0]), len(manasi_test['1']['graph_dicts'][0][0])

In [ ]:
# here is the graph info of the first frame in the sequence
manasi_test['1']['graph_dicts'][0][0]

In [ ]:
len(manasi_test['1']['sequences']), type(manasi_test['1']['sequences'])

In [ ]:
# in this sequence there are 389 frames
manasi_test['1']['sequences'][0].shape, type(manasi_test['1']['sequences'][0])

In [ ]:
manasi_test['1']['sequences'][0][0].shape, type(manasi_test['1']['sequences'][0][0])

In [ ]:
# here is the entity info of the first frame
manasi_test['1']['sequences'][0][0]

In [ ]:
type(manasi_test['1']['sequences'][0][0][0][0])

In [ ]:
# labels of this video
manasi_test['1']['labels']

In [ ]:
# each is the label of the sequence
manasi_test['12']['labels']

dict of video names --> sequence --> sequences --> frames --> [0] frame feature (np array of lists, each list is a 25 dimension of feature)[1] frame label

In [ ]:
manasi_test['1']['sequences'][0][0][0]

In [ ]:
(manasi_test['1']['sequences'][0][0].shape)

In [ ]:
manasi_test['1']['sequences'][0][0][0][0]

In [ ]:
manasi_test['1']['sequences'][0][0]

In [ ]:
manasi_test['1']['sequences'][0][0][1]


In [ ]:
np.array([1, 2, 3])

dict of all videos --> dict of ['graph_dicts', 'sequences', 'labels'] for each video -->

graph_dicts list of all sequences' graph dicts --> list of graph dicts for each frame --> 


In [ ]:
load_pickle('GAZE_dataset/preprocess/video_data/1.mp4/sequence_0')

## Build Graph


In [5]:
def process_sequence(sequence_annotations):
    
    all_labels = ['Person1', 'Person2', 'Person3', 'Person4', 'Object1', 'Object2', 'Object3', 'Object4', 'Object5']
    grouped = sequence_annotations.groupby('frame_numbers')
    graph_dicts_frames = []
    for frame_number, group in grouped:
        senders, receivers, nodes = [], [], []

        # Process in the order of the all_labels list
        entities = []
        for entity in all_labels:                
            if (group['labels'] == entity).any():
                entities.append(entity)
                
        for i, entity in enumerate(entities):
            # Access the value in the 'features' and 'gazes' columns of that entity
            feature = group.loc[group['labels'] == entity, 'features'].iloc[0].tolist()
            nodes.append(feature)
            edge = group.loc[group['labels'] == entity, 'gaze_direction'].iloc[0]
            # if gaze exists (only when the entity is person)
            if not isinstance(edge, float):
                sender, receiver = edge.split(', ')
                # if the gaze is at some entities not found in the video, the gaze will be discarded
                if sender == entity and receiver in entities: 
                    senders.append(i)
                    receivers.append(entities.index(receiver))
                # else:
                #     print(f'unrecognised gaze {edge} in {entities}')
        while len(nodes) < 5:
            nodes.append([0 for _ in range(25)])
    
        assert len(nodes) >= 5
        graph_dict = {'nodes': nodes, 'senders': senders, 'receivers': receivers}
        graph_dicts_frames.append(graph_dict)
    return graph_dicts_frames

In [6]:
all_videos = {}
for video_name in tqdm(sorted(os.listdir(patches_path))):
    
    video_dir = os.path.join(patches_path, video_name)
    video_data = {'graph_dicts': [],
                  # 'sequences': [],
                  'labels': load_pickle(os.path.join(video_dir, 'labels'))}
    graph_dicts_sequences = []
    
    for pickle_name in os.listdir(video_dir):
        if 'sequence_' in pickle_name:
            sequence_annotations = load_pickle(os.path.join(video_dir, pickle_name))
            graph_dicts_sequences.append(process_sequence(sequence_annotations))
    video_data['graph_dicts'] = graph_dicts_sequences
    all_videos[video_name.replace('.mp4', '')] = video_data

 95%|█████████▌| 285/299 [04:00<00:11,  1.19it/s]

KeyboardInterrupt



In [ ]:
save_pickle(all_videos, "GAZE_dataset/preprocess/graphs")

In [ ]:
all_videos = load_pickle("GAZE_dataset/preprocess/graphs")

In [ ]:
def check(data1, data2):
    import random

    # Generate 10 random integers
    random_integers = [str(random.randint(1, 299)) for _ in range(100)]
    
    assert list(data1.keys()) == list(data2.keys())
    for i in tqdm(random_integers):
        if i in ['255', '291']:
            continue
        if data1[i]['labels'] != data2[i]['labels']:
            print(f'{i} labels do not match')
        if len(data1[i]['graph_dicts']) != len(data2[i]['graph_dicts']):
            print(f'{i} graph_dicts do not match')
        if len(data1[i]['graph_dicts'][0]) != len(data2[i]['graph_dicts'][0]): 
            print(f'{i} graph_dicts[0] do not match')
        assert len(data1[i]['graph_dicts'][0][0]) == len(data2[i]['graph_dicts'][0][0])
    
    random_int = str(random.randint(1, 299))
    print(f'========== check the following {random_int}')
    print(data1[random_int]['graph_dicts'][0][0])
    print(data2[random_int]['graph_dicts'][0][0])
        

In [ ]:
check(V, all_videos)

In [ ]:
V.keys()

In [8]:
for node in V['1']['graph_dicts'][0][0]['nodes']:
    print(node)

[19.127209432632547, -1.0256831867282694, 8.13335232314885, -4.201163709443032, 1.8991420331250286, -13.876740169041135, -5.526908531131593, -3.8356414133755274, 9.519650781813894, -3.851486033541, -10.705018150235949, -5.2528136370278915, -1.2222795299178517, 2.5200896122278476, -1.901773983292112, -0.8833180093497349, -0.5375356957939621, -4.3363662521959885, 3.3842700418425578, -3.2342044946744766, 221, 329, 222, 281, 1]
[-10.227210662910501, -7.964693699370105, 12.540352411872792, -11.681789480086213, 0.30675131708999126, -10.135767468020056, -5.339033053269009, -1.821363372032125, 5.746255099644273, -0.02389578031439287, -6.916765072423783, 8.810839903650729, 1.9119219721072658, 3.520538773271611, 6.203606830129152, 7.181162217732476, -3.5985563941815815, -0.4146291782533541, 6.5097611640900555, 1.183150324798416, 54, 225, 104, 219, 0]
[-12.812543674851636, -16.574032737620364, 5.022561399135988, -1.9710347021824894, -4.590026455200244, -14.132404203224567, 3.5610422346674504, -2.

In [9]:
for node in all_videos['1']['graph_dicts'][0][0]['nodes']:
    print(node)

[10.227222442626953, 7.964751243591309, 12.540274620056152, -11.68178653717041, 0.30670633912086487, 10.135627746582031, -5.338925838470459, -1.823194146156311, -5.744480609893799, 0.027499111369252205, -6.921828269958496, -8.80478572845459, 1.9131251573562622, -3.541015863418579, 6.224554061889648, 7.135051250457764, 3.634446144104004, -0.4626026749610901, 6.491567611694336, 1.4666532278060913, 54.0, 225.0, 104.0, 219.0, 0.0]
[12.812554359436035, 16.574092864990234, 5.022436141967773, -1.971082329750061, -4.589867115020752, 14.132447242736816, 3.5617623329162598, -2.1517295837402344, -9.754671096801758, 5.933873176574707, 7.9097981452941895, 4.132099151611328, 2.4401583671569824, -5.6511383056640625, 4.711540699005127, 1.6954163312911987, 3.5094573497772217, -1.1490932703018188, 0.43998879194259644, -2.007664918899536, 46.0, 213.0, 402.0, 533.0, 0.0]
[-19.127185821533203, 1.0257126092910767, 8.133258819580078, -4.2011542320251465, 1.8991039991378784, 13.876609802246094, -5.52685260772

In [24]:
V = {}
with open('GAZE_dataset/Manasi_preprocessed_pickles/processed_Mar25_0', 'rb') as f:
    V.update(pickle.load(f))

with open('GAZE_dataset/Manasi_preprocessed_pickles/processed_Mar25_100', 'rb') as f:
    V.update(pickle.load(f))

with open('GAZE_dataset/Manasi_preprocessed_pickles/processed_Mar25_200', 'rb') as f:
    V.update(pickle.load(f))


myKeys = list(V.keys())
myKeys.sort()
V = {i: V[i] for i in myKeys}

In [25]:
from collections import Counter
bootstrapping = 10
from sklearn.model_selection import train_test_split
mode = 'standard'

In [27]:
def check_label_anomalies(seq_labels):
	counts = Counter(seq_labels)
	if 'JointAtt' in counts.keys() and counts['JointAtt'] == 1:
		return False
	elif 'MutualGaze' in counts.keys() and counts['MutualGaze'] == 1:
		return False
	elif 'NA' in counts.keys() and counts['NA'] >= 1:
		return False
	else:
		return True
errors = []
for i in range(bootstrapping):
    ### Split Videos into train and test (stratify not possible because will have to separate labels then)
    V_train_idx, V_test_idx = train_test_split(list(V.keys()), random_state=123+i) #5Jun = 13+i, 14Jun = 27+i, 14eveJun = 123 +i
    Sequences = []
    seq_train_idx = []
    seq_test_idx = []
    for k in V_train_idx:
        # print(V_train_idx)
        # print(k)
        seq_labels = [tuple(v for k,v in seq.items() if k[0]=='P') for seq in V[k]['labels']]
        for ind, seq in enumerate(seq_labels):
            if check_label_anomalies(seq) and len(seq)>=2 and len(V[k]['labels'][ind])<=5:  #remove this last condition later
                Seq_dict = dict()
                Seq_dict['label'] = seq[::-1]  # reversing RETHINK THIS
				# reverse node features (keeping last two persons ahead) ##RETHINK THIS
                for frame_id in range(len(V[k]['graph_dicts'][ind])):
                    node_features = np.array(V[k]['graph_dicts'][ind][frame_id]['nodes'])
                    V[k]['graph_dicts'][ind][frame_id]['nodes'] = node_features[::-1].tolist()
    # 
    #                 if mode == "standard":
                    padding = np.zeros((5-len(node_features),node_features.shape[1])).tolist()
                    V[k]['graph_dicts'][ind][frame_id]['nodes'].extend(padding)

                    
                Seq_dict['graph_dicts'] = V[k]['graph_dicts'][ind]
                for j in range(len(Seq_dict['graph_dicts'])):
                    if Seq_dict['graph_dicts'][j]['nodes'][0] == [0 for _ in range(25)]:
                        # print(seq_labels)
                        errors.append([i, k, ind, j])
            
                    
                # seq_train_idx.append(len(Sequences))
                # Sequences.append(Seq_dict)
        #     break
        # break
    #   
	# for k in V_test_idx:
	# 	seq_labels = [tuple(v for k,v in seq.items() if k[0]=='P') for seq in V[k]['labels']]
	# 	for ind, seq in enumerate(seq_labels):
	# 		if check_label_anomalies(seq) and len(seq)>=2 and len(V[k]['labels'][ind])<=5:  #remove this last condition later
	# 			Seq_dict = dict()
	# 			Seq_dict['label'] = seq[::-1]  # reversing RETHINK THIS
    # 
	# 			# reverse node features (keeping last two persons ahead) ##RETHINK THIS
	# 			for frame_id in range(len(V[k]['graph_dicts'][ind])):
	# 				node_features = np.array(V[k]['graph_dicts'][ind][frame_id]['nodes'])
	# 				V[k]['graph_dicts'][ind][frame_id]['nodes'] = node_features[::-1].tolist()
    # 
	# 				if mode == "standard":
	# 					padding = np.zeros((5-len(node_features),node_features.shape[1])).tolist()
	# 					V[k]['graph_dicts'][ind][frame_id]['nodes'].extend(padding)
    # 
	# 			Seq_dict['graph_dicts'] = V[k]['graph_dicts'][ind]
    # 
	# 			seq_test_idx.append(len(Sequences))
	# 			Sequences.append(Seq_dict)
    # 
	# print("Train Seqs", len(seq_train_idx), "Test Seqs", len(seq_test_idx))


In [28]:
errors[0]

[1, '15', 0, 0]

In [16]:
V['37']['graph_dicts'][1][134]

{'nodes': [[-3.0686977176502137,
   -16.462693322343235,
   12.819795932345222,
   -3.1687428067800476,
   -1.0203585893875724,
   5.682883175930007,
   -2.2786209917925815,
   -5.218296999587048,
   4.848294112234644,
   -0.4699461901103268,
   6.557035838865493,
   -0.5260438243443568,
   -7.303968359671414,
   0.9738747281569077,
   0.2538767715917672,
   -1.8761696228337952,
   1.8352606381298984,
   -4.117292776096059,
   0.5040785413039112,
   -3.609829107573127,
   33.0,
   107.0,
   385.0,
   448.0,
   0.0],
  [11.906825422187474,
   -11.88630108407816,
   11.403745932411425,
   5.200955818868563,
   8.435605397522327,
   -0.42978243503074176,
   -9.313480966028855,
   13.25618719898381,
   7.87509784261819,
   -3.829458400099795,
   3.84527633095601,
   -5.525571142829749,
   -9.939142198350574,
   0.4804799593785418,
   -4.396122704360587,
   1.092364898776514,
   -1.353065360662791,
   -5.479873354656828,
   7.082500051582039,
   -3.7794243579538302,
   36.0,
   107.0,
   30

In [ ]:
len(Sequences)

In [ ]:
type(Sequences[0]), len(Sequences[0]), Sequences[0].keys()

In [ ]:
Sequences[0]['label']

In [ ]:
len(Sequences[0]['graph_dicts'])

In [ ]:
len(Sequences[0]['graph_dicts'][0]['nodes'])

In [ ]:
Sequences[-3]['graph_dicts'][0]['nodes']

In [ ]:
s = load_pickle('GAZE_dataset/manasi_bootstrapped/traintest_seqs_5Jun23_9')

In [ ]:
len(s)

In [ ]:
type(s), len(s)

In [ ]:
type(s[0]), len(s[0]), s[0].keys()

In [ ]:
type(s[1]['graph_dicts'])

In [ ]:
s[1]['graph_dicts'][0]['nodes'][0]

In [17]:
for f in os.listdir('GAZE_dataset/manasi_bootstrapped'):
    file_name = os.path.join('GAZE_dataset/manasi_bootstrapped', f)
    idx = "_".join(file_name.split('_')[-2:])
    Sequence = load_pickle(file_name)
    print(file_name)
    errors = []
    for i in range(len(Sequence)):
        for j in range(len(Sequence[i]['graph_dicts'])):
            if Sequence[i]['graph_dicts'][j]['nodes'][0] == [0 for _ in range(25)]:
                # print(seq_labels)
                errors.append([idx, i, j])
        

GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_0
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_1
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_2
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_3
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_4
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_5
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_6
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_7
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_8
GAZE_dataset/manasi_bootstrapped\traintest_seqs_14Jun23_9
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_0
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_1
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_2
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_3
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_4
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_5
GAZE_dataset/manasi_bootstrapped\traintest_seqs_5Jun23_6
GAZE_dataset/manasi_b

In [ ]:
len(errors)

In [29]:
boot_set = None
for error in errors:
    if error[0] != boot_set:
        print(error[0])
        boot_set = error[0]

1
2
3
4
5
6
7
8
9


In [ ]:
errors[0]

In [ ]:
column_index = 1
unique_values = set(row[column_index] for row in errors)

# Converting the set back to a list if needed
unique_values = list(unique_values)

print(unique_values)

In [32]:
s = load_pickle('GAZE_dataset/manasi_bootstrapped/traintest_seqs_5Jun23_9')

for feat in s[1]['graph_dicts'][0]['nodes']:
    print(feat)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-13.818248182769008, -10.476121501397772, 3.368019906217009, -9.782118738309396, -4.510680902946422, -1.1619046727240887, 3.5071690343794746, -6.323518275572182, -3.9789037548364092, -9.397165925527396, 9.796443549520045, 0.15798833771417098, -5.717554018561345, 1.1535469727325642, 1.9779028051724334, -5.4560061613642095, -3.955755922377748, -4.181292693076908, -13.769082497516512, -3.9039636901543795, 72.0, 259.0, 148.0, 300.0, 0.0]
[-9.746132097891433, -11.380606163767357, -8.166118808729234, 2.552620620316964, -5.049897747516092, 1.4793527744497919, -1.5685977208771757, -0.6680178920360503, -5.550115972844827, 

In [ ]:
s[2]['graph_dicts'][1]['nodes']


Check how many videos have more than 2 people

In [9]:
annot = load_pickle('Data/preprocess/video_data/-YwZOeyAQC8_15.mp4/annotations')

In [10]:
annot

,labels,gazes,frame_numbers,left,right,top,bottom,features
0,head1,"head1, neither",1,0,94,101,228,"[6.680127143859863, 3.3551645278930664, -3.418..."
1,head2,"head2, object1",1,86,155,211,286,"[14.151459693908691, 2.1622977256774902, 6.103..."
2,object1,NaN,1,189,243,279,327,"[19.33083724975586, -5.286369800567627, -12.16..."
3,head1,"head1, neither",2,0,93,101,225,"[6.7137017250061035, 2.192903757095337, -6.567..."
4,head2,"head2, object1",2,86,154,211,285,"[15.673377990722656, 2.015958786010742, 4.1520..."
...,...,...,...,...,...,...,...,...
255,head2,"head2, neither",89,81,149,204,276,"[20.822486877441406, 6.350839138031006, 5.5030..."
256,object1,NaN,89,151,246,257,314,"[20.562641143798828, -0.4833153784275055, -10...."
257,head1,"head1, neither",90,4,84,98,220,"[13.18536376953125, 0.4746054410934448, -11.00..."
258,head2,"head2, neither",90,79,147,205,277,"[20.570606231689453, 6.350813388824463, 5.0889..."


In [27]:
video_dir = 'Data/preprocess/video_data'
not_valid_videos ={}
for video_name in os.listdir(video_dir):
    video_path = os.path.join(video_dir, video_name)
    annot = load_pickle(os.path.join(video_path, 'annotations'))
    grouped = annot.groupby('frame_numbers')
    for frame_number, group in grouped:
        
        if (group['labels'] == 'head1').any() and (group['labels'] == 'head2').any():
            continue
        else:
            if video_name not in not_valid_videos:
                not_valid_videos[video_name] = [frame_number]
            else:
                not_valid_videos[video_name].append(frame_number)

In [28]:
len(not_valid_videos)

35

In [31]:
for key, value in not_valid_videos.items():
    print(key, value)

-YwZOeyAQC8_15.mp4 [58, 59, 60, 61, 70, 71, 72, 73, 86, 87]
b7y36KyAIoI_120.mp4 [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
FJzKgF-nR0E_15.mp4 [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
flickr-4-1-6-2-6-0-0-0-23041626000_1.mp4 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
flickr-4-1-7-8-1-9-6-2-3941781962_7.mp4 [62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
flickr-4-7-1-4-7-6-4-0-2447147640_28.mp4 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
flickr-4-8-3-9-9-3-0-9-6748399309_20.mp4 [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46